In [1]:
pip install wget


  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=3454295bb3c82b23a0ad5ecede6e5297fab34b11df8be5a68b448922621444fc
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [4]:
!pip install tensorflow-gpu==2.0.rc

     |████████████████████████████████| 380.5MB 43kB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 501kB 38.4MB/s 
     |████████████████████████████████| 4.3MB 25.6MB/s 


In [2]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
pwd

'/content'

In [5]:
cd /content/drive/MyDrive/ML

/content/drive/MyDrive/ML


In [13]:
link = "https://www.kaggle.com/vbookshelf/rice-leaf-diseases.zip"

In [11]:
import wget

In [15]:
data = "./"

In [16]:
!unzip -q archive.zip -d $data

In [6]:
pip install vit-keras


     |████████████████████████████████| 686kB 3.0MB/s 


In [7]:
import os
import cv2
import numpy as np
import random
from glob import glob
from tqdm.auto import tqdm


In [9]:
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
import albumentations as albu
from albumentations import Compose, ShiftScaleRotate, Resize
from albumentations.pytorch import ToTensor


from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.metrics import confusion_matrix
import itertools
from sklearn.metrics import classification_report

import shutil

In [11]:
image_height = 224
image_width = 224
image_channels = 3

In [22]:
pwd

'/content/drive/My Drive/ML'

In [12]:
leaf_smut_list =  os.listdir("/content/drive/MyDrive/ML/rice_leaf_diseases/Leaf smut")

In [13]:
brown_spot_list = os.listdir("/content/drive/MyDrive/ML/rice_leaf_diseases/Brown spot")
bacterial_leaf_blight = os.listdir("/content/drive/MyDrive/ML/rice_leaf_diseases/Bacterial leaf blight")

In [14]:
print(len(leaf_smut_list))
print(len(brown_spot_list))
print(len(bacterial_leaf_blight))

40
40
40


In [15]:
##Create the train and val sets

In [16]:
import pandas as pd


In [17]:
df_leaf_smut = pd.DataFrame(leaf_smut_list, columns=['image'])
df_leaf_smut['target'] = 'leaf_smut'

df_brown_spot = pd.DataFrame(brown_spot_list, columns=['image'])
df_brown_spot['target'] = 'brown_spot'

df_bacterial_leaf_blight = pd.DataFrame(bacterial_leaf_blight, columns=['image'])
df_bacterial_leaf_blight['target'] = 'bacterial_leaf_blight'

In [18]:
##Create val set for each class

In [19]:
# Sample 5 validation images from each class
df_leaf_smut_val = df_leaf_smut.sample(n=5, random_state=101)
df_brown_spot_val = df_brown_spot.sample(n=5, random_state=101)
df_bacterial_leaf_blight_val = df_bacterial_leaf_blight.sample(n=5, random_state=101)

In [20]:
print(len(df_leaf_smut_val))
print(len(df_brown_spot_val))
print(len(df_bacterial_leaf_blight_val))

5
5
5


In [21]:
df_leaf_smut_val.describe

<bound method NDFrame.describe of            image     target
18  DSC_0336.jpg  leaf_smut
16  DSC_0335.JPG  leaf_smut
36  DSC_0505.jpg  leaf_smut
25  DSC_0511.jpg  leaf_smut
26  DSC_0516.jpg  leaf_smut>

In [22]:
##Create the train set for each class

#get a list of val images
val_list = list(df_leaf_smut_val['image'])
#filter out the val images
df_leaf_smut_train = df_leaf_smut[~df_leaf_smut['image'].isin(val_list)]

# brown_spot
# get a list of val images
val_list = list(df_brown_spot_val['image'])
# filter out the val images
df_brown_spot_train = df_brown_spot[~df_brown_spot['image'].isin(val_list)] # ~ means notin

# bacterial_leaf_blight
# get a list of val images
val_list = list(df_bacterial_leaf_blight_val['image'])
# filter out the val images
df_bacterial_leaf_blight_train = \
df_bacterial_leaf_blight[~df_bacterial_leaf_blight['image'].isin(val_list)] # ~ means notin

In [23]:
print(len(df_leaf_smut_train))
print(len(df_brown_spot_train))
print(len(df_bacterial_leaf_blight_train))

35
35
35


**Create df_data, df_train, df_val**

In [25]:
df_data = pd.concat([df_leaf_smut, df_brown_spot, df_bacterial_leaf_blight], axis=0).reset_index(drop=True)

df_train = \
pd.concat([df_leaf_smut_train, df_brown_spot_train, df_bacterial_leaf_blight_train], axis=0).reset_index(drop=True)

In [26]:
df_val = \
pd.concat([df_leaf_smut_val, df_brown_spot_val, df_bacterial_leaf_blight_val], axis=0).reset_index(drop=True)

In [28]:
df_data = shuffle(df_data)
df_train = shuffle(df_train)
df_val = shuffle(df_val)


In [29]:
print(df_data.shape)
print(df_train.shape)
print(df_val.shape)


(120, 2)
(105, 2)
(15, 2)


In [30]:
df_data['target'].value_counts()

leaf_smut                40
bacterial_leaf_blight    40
brown_spot               40
Name: target, dtype: int64

In [31]:
df_train['target'].value_counts()

leaf_smut                35
bacterial_leaf_blight    35
brown_spot               35
Name: target, dtype: int64

In [32]:
df_val['target'].value_counts()

leaf_smut                5
bacterial_leaf_blight    5
brown_spot               5
Name: target, dtype: int64

In [33]:
# Create the target as index values

# combine val, train and test
val_len = len(df_val)
train_len = len(df_train)
df_combined =  pd.concat(objs=[df_val, df_train], axis=0).reset_index(drop=True)

# create the dummy variables
df_combined = pd.get_dummies(df_combined, columns=['target'])

# separate the train and val sets
df_val = df_combined[:val_len]
df_train = df_combined[val_len:]


In [34]:
print(df_val.shape)
print(df_train.shape)

(15, 4)
(105, 4)


In [35]:
df_combined.head()

,image,target_bacterial_leaf_blight,target_brown_spot,target_leaf_smut
0,DSC_0303.JPG,0,1,0
1,DSC_0333.JPG,0,1,0
2,DSC_0372.JPG,1,0,0
3,DSC_0305.JPG,0,1,0
4,DSC_0505.jpg,0,0,1


In [36]:
df_train.head()

,image,target_bacterial_leaf_blight,target_brown_spot,target_leaf_smut
15,DSC_0316.JPG,0,0,1
16,DSC_0514.jpg,0,0,1
17,DSC_0507.jpg,0,0,1
18,DSC_0399.JPG,1,0,0
19,DSC_0502.jpg,0,0,1


In [37]:
df_val.head()

,image,target_bacterial_leaf_blight,target_brown_spot,target_leaf_smut
0,DSC_0303.JPG,0,1,0
1,DSC_0333.JPG,0,1,0
2,DSC_0372.JPG,1,0,0
3,DSC_0305.JPG,0,1,0
4,DSC_0505.jpg,0,0,1
